COLLAB SETUP

In [ ]:
#from google.colab import drive
#drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
import tensorflow as tf
print(tf.keras.__version__)

2.9.0


IMPORTING LIBRARIES

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import VGG16
import cv2
from tensorflow.keras.models import load_model
import numpy as np

IMPORTING TRAINING DATASET

In [ ]:
image_generator=ImageDataGenerator(vertical_flip=False,horizontal_flip=True,shear_range=0.1,zoom_range=0.1,rescale=1/255,brightness_range=(0.2,0.8))
X_train1=image_generator.flow_from_directory(target_size=(224,224),
                                    directory="drive/My Drive/Project Data/body/training",
                                    class_mode="categorical",
                                   batch_size=10,
                                            subset="training")

IMPORTING TESTING DATASET

In [ ]:
image_generator_1=ImageDataGenerator(vertical_flip=False,horizontal_flip=True,shear_range=0.1,zoom_range=0.1,rescale=1/255,brightness_range=(0.2,0.8))
X_test1=image_generator_1.flow_from_directory(target_size=(224,224),
                                    directory="drive/My Drive/Project Data/body/validation",
                                    class_mode="categorical",
                                   batch_size=10,
                                )

Found 171 images belonging to 3 classes.


INTTIALZE MODEL

In [ ]:
vgg16=VGG16(include_top=False,input_shape=(224,224,3),weights='imagenet')
for i in vgg16.layers:
  i.trainable=False

ADD FLATTEN LAYER

In [ ]:
flatten_layer=Flatten()(vgg16.output)


ADD DENSE LAYER

In [ ]:
dense32=Dense(32,kernel_initializer=RandomNormal,activation="relu")(flatten_layer)
output=Dense(3,activation="softmax")(dense32)

CREATE MODEL

In [ ]:
model1=Model(inputs=vgg16.input,outputs=output)


CONFIGURE LEARNING PROCESS

In [ ]:
model1.compile(loss=CategoricalCrossentropy(),
              optimizer="adam",
              metrics=["accuracy"])

TRAIN THE MODEL

In [ ]:
model1.fit(X_train1,validation_data=X_test1,epochs=25,steps_per_epoch=97,validation_steps=17)

Epoch 1/25
97/97 [==============================] - 584s 6s/step - loss: 1.5552 - accuracy: 0.4727 - val_loss: 1.0637 - val_accuracy: 0.5941
Epoch 2/25
97/97 [==============================] - 575s 6s/step - loss: 0.9066 - accuracy: 0.6171 - val_loss: 1.1633 - val_accuracy: 0.5471
Epoch 3/25
97/97 [==============================] - 576s 6s/step - loss: 1.0237 - accuracy: 0.5511 - val_loss: 0.9585 - val_accuracy: 0.5471
Epoch 4/25
97/97 [==============================] - 577s 6s/step - loss: 0.8211 - accuracy: 0.5955 - val_loss: 0.9118 - val_accuracy: 0.5765
Epoch 5/25
97/97 [==============================] - 572s 6s/step - loss: 0.7611 - accuracy: 0.6244 - val_loss: 0.9034 - val_accuracy: 0.5529
Epoch 6/25
97/97 [==============================] - 568s 6s/step - loss: 0.7301 - accuracy: 0.6605 - val_loss: 0.9219 - val_accuracy: 0.5588
Epoch 7/25
97/97 [==============================] - 569s 6s/step - loss: 0.6918 - accuracy: 0.6832 - val_loss: 0.8403 - val_accuracy: 0.5941
Epoch 8/25
97

SAVING THE MODEL

In [ ]:
model1.save("Model1.h5")

In [ ]:
model = load_model("Model1.h5")


def detect(frame):
    img = cv2.resize(frame, (224, 224))
    if(np.max(img) > 1):
        img = img/255.0
    img = np.array([img])
    prediction = model.predict(img)
    label = ["front", "rear", "side"]
    preds = label[np.argmax(prediction)]
    return preds